<a href="https://colab.research.google.com/github/ballmdr/Forex-Prediction-Machine_Learning/blob/master/Forex_Pytorch_RNN_LSTM_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install fxcmpy

In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
import fxcmpy
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

from google.colab import drive
drive.mount('/content/drive')

def connect():
    return fxcmpy.fxcmpy(access_token=access_token, server=server)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
access_token = '82e41ea2bcf020754f67aeb835e9ad617dbc5427'
server = 'demo'
symbol = 'BTCUSD'
symbol2 = 'BTC/USD'
target_windows = 5
digits = 1
ratio = 100
n_prices = 10000

In [0]:
con = connect()

|WARNING|2019-07-29 02:31:24,411|Default account set to 1041561, to change use set_default_account().


In [0]:
def dfProcess(df):
# Short-Term
# ใช้ค่า 3, 5, 8, 10, 12, or 15.
# Long-Term
# ใช้ค่า 30, 35, 40, 45, 50, or 60.
  df['ma3'] = np.round(df.close.rolling(3).mean(), 5)
  df['ma5'] = np.round(df.close.rolling(5).mean(), 5)
  df['ma8'] = np.round(df.close.rolling(8).mean(), 5)
  df['ma10'] = np.round(df.close.rolling(10).mean(), 5)
  df['ma12'] = np.round(df.close.rolling(12).mean(), 5)
  df['ma15'] = np.round(df.close.rolling(15).mean(), 5)
  #df['ma20'] = np.round(df.close.rolling(20).mean(), 5)
  #df['ma7_shift3'] = df.ma7.shift(-3)
  #df['ma10_shift3'] = df.ma10.shift(-3)
  #df['close_shift3'] = df.close.shift(-3)
  df['close_diff_pip'] = - df.close.diff(-target_windows) * digits
  
  arr = ['0', '1', '2', '3', '4']
  for i in range(1, target_windows):
    df[arr[i]] = - df.close.diff(-i) * digits
    
  df.dropna(inplace=True)

  #df['close_target_percent'] = np.round((df['close_diff_pip']*digits)/df.close, 2) * 100
  #df['7_10_target_percent'] = np.round(((df['ma7_shift3']-df['ma10_shift3'])*digits)/df['ma10_shift3'], 2)
  #df['5_7_target_percent'] = np.round(((df.ma5-df.ma7)*digits)/df.ma7, 2)
  

  df['Target'] = np.ones(len(df))
  # df['Target'].loc[(df['close_target_percent'] > 9) & (df['7_10_target_percent'] > 1) & (df['5_7_target_percent'] > 1)] = 2
  # df['Target'].loc[(df['close_target_percent'] < -9) & (df['7_10_target_percent'] < -1) & (df['5_7_target_percent'] < -1)] = 0
  df['Target'].loc[(df['close_diff_pip'] > ratio) &
                   (df['1'] > -ratio) &
                   (df['2'] > -ratio) &
                   (df['3'] > -ratio) &
                   (df['4'] > -ratio)
                  ] = 2
  df['Target'].loc[(df['close_diff_pip'] < -ratio) &
                   (df['1'] < ratio) &
                   (df['2'] < ratio) &
                   (df['3'] < ratio) &
                   (df['4'] < ratio)                    
                  ] = 0
  print(df.Target.value_counts())

  df.drop(['bidopen', 'bidclose', 'bidhigh', 'askopen', 'askclose', 'askhigh', 'bidlow', 'asklow', 'tickqty', 'close_diff_pip', '1', '2', '3', '4'], axis=1, inplace=True)
  df.dropna(inplace=True)
  print(df.head())
  print(df.shape)
  
  print(df.info())
  
  return df


def getXY(df, windows):

  X = []
  Y = []
  for i in range(len(df)-(windows+1)):
    

    fig = plt.figure(frameon=False) 
    fig.set_size_inches(3,2)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)

    _df = df.iloc[i:windows]
    plt.plot(_df.ma3, color='red')
    plt.plot(_df.ma5, color='blue')
    plt.plot(_df.ma8, color='green')
    plt.plot(_df.ma10, color='orange')
    plt.plot(_df.ma12, color='black')
    plt.plot(_df.ma15, color='purple')
    #plt.plot(_df.open, color='black')
   # plt.plot(_df.close, color='purple')

    plt.savefig('tmp2.png')

    plt.close()

    img = plt.imread('tmp2.png')
    new_img = resize(img, (100,150))
    #print(new_img.shape)
    if i % 1000 == 0:
      print(i)
      plt.imshow(new_img)
      plt.show()
      print(i, ' - ', windows)
      print('target: ', df.Target.iloc[windows-1])
    X.append(new_img)
    Y.append(df.Target.iloc[windows-1])

    windows += 1

  return X, Y


In [0]:
df = con.get_candles(symbol2, period='H4', number=n_prices)
df['close'] = (df.bidclose + df.askclose) / 2
df = dfProcess(df)

#df['close'] = (df.close - df.close.mean()) / (df.close.max() - df.close.min())
#print(df['close'].head())

split = int(len(df) * 0.60)
train_data = df.iloc[:split].drop(['Target'], axis=1).values
train_target = df.iloc[:split]['Target'].values
test_data = df.iloc[split:].drop(['Target'], axis=1).values
test_target = df.iloc[split:]['Target'].values


minmax = MinMaxScaler()
train_data = minmax.fit_transform(train_data)
test_data = minmax.transform(test_data)
print(train_data.shape)
print(train_target.shape)
print(test_data.shape)
print(test_target.shape)

1.0    925
2.0    390
0.0    327
Name: Target, dtype: int64
                      close         ma3     ma5  ...        ma12        ma15  Target
date                                             ...                                
2018-06-27 15:00:00  6105.0  6097.00000  6107.2  ...  6183.95833  6180.70000     1.0
2018-06-27 19:00:00  6142.5  6118.50000  6102.7  ...  6171.87500  6180.86667     1.0
2018-06-28 00:00:00  6135.0  6127.50000  6113.7  ...  6159.95833  6180.20000     1.0
2018-06-28 04:00:00  6132.5  6136.66667  6124.6  ...  6150.33333  6174.50000     0.0
2018-06-28 08:00:00  6110.0  6125.83333  6125.0  ...  6138.83333  6162.66667     0.0

[5 rows x 8 columns]
(1642, 8)
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1642 entries, 2018-06-27 15:00:00 to 2019-07-26 05:00:00
Data columns (total 8 columns):
close     1642 non-null float64
ma3       1642 non-null float64
ma5       1642 non-null float64
ma8       1642 non-null float64
ma10      1642 non-null float64
ma12      1

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [0]:
train_data[0:5]

array([[1.60214413e-02, 5.87354478e-04, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 4.12435766e-01,
        3.29394387e-01, 3.92737430e-01, 5.81753890e-01, 5.34142640e-01],
       [0.00000000e+00, 0.00000000e+00, 2.83960092e-02, 1.73719686e-02,
        1.41249952e-02, 1.22750490e-02, 1.06621397e-02, 4.70914697e-01,
        4.22451994e-01, 5.65921788e-01, 6.10183876e-01, 5.24911482e-01],
       [2.15008934e-02, 2.67347187e-03, 6.00222220e-02, 3.71134737e-02,
        3.00211323e-02, 2.60192593e-02, 2.24959834e-02, 4.73175745e-01,
        5.12555391e-01, 5.06005587e-01, 5.10749646e-01, 5.17197774e-01],
       [7.93329363e-02, 2.42030210e-02, 7.49475951e-02, 6.34632757e-02,
        5.11644845e-02, 4.38322699e-02, 3.69177968e-02, 4.36690647e-01,
        3.59084195e-01, 3.22625698e-01, 4.15841584e-01, 4.42842691e-01],
       [7.52829065e-02, 4.98035411e-02, 8.68031294e-02, 8.94725870e-02,
        7.18222899e-02, 6.11509810e-02, 5.13213601e-02, 3.33

In [0]:
df.iloc[0:5]

,close,ma3,ma5,ma8,ma10,ma12,ma15,close_diff_pip,1,2,3,4,Target
date,,,,,,,,,,,,,
2019-04-02 10:00:00,4810.00,4579.66667,4402.45,4298.81250,4260.050,4231.72917,4192.33333,176.25,10.00,-57.25,102.75,275.75,2.0
2019-04-02 14:00:00,4820.00,4796.66667,4538.90,4385.34375,4332.825,4291.89583,4246.36667,163.25,-67.25,92.75,265.75,166.25,2.0
2019-04-02 18:00:00,4752.75,4794.25000,4662.35,4464.71875,4396.325,4347.77083,4295.05000,305.50,160.00,333.00,233.50,230.50,2.0
2019-04-02 23:00:00,4912.75,4828.50000,4811.10,4562.21875,4474.825,4416.14583,4351.08333,241.50,173.00,73.50,70.50,145.50,2.0
2019-04-03 03:00:00,5085.75,4917.08333,4876.25,4680.71875,4571.625,4496.81250,4416.93333,222.25,-99.50,-102.50,-27.50,68.50,1.0


In [0]:
train_data[0:5][0].reshape(1,-1).shape

(1, 12)

In [0]:
class RNN(nn.Module):
    def __init__(self, hidden, K, input_size):
        super(RNN, self).__init__()
        
        self.linearH = nn.Linear(input_size + hidden, hidden)
        self.linearO = nn.Linear(input_size + hidden, K)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x, h):
        x = torch.cat((x, h), 1)
        h = torch.tanh(self.linearH(x))
        out = self.softmax(self.linearO(x))
        
        return out, h
      
      
class LSTM(nn.Module):
  def __init__(self, hidden, output_size, vocab_size, n_layers):
    super(LSTM, self).__init__()
    self.n_layers = n_layers
    
    self.linearFt = nn.Linear(vocab_size + hidden*2, hidden)
    self.linearIt = nn.Linear(vocab_size + hidden*2, hidden)
    self.linearCt1 = nn.Linear(vocab_size + hidden*2, hidden)
    self.linearOt = nn.Linear(vocab_size + hidden*2, hidden)
    self.linearO = nn.Linear(vocab_size + hidden*2, output_size)
    self.drop = nn.Dropout(p=0.50)
    self.softmax = nn.LogSoftmax(dim=1)
    
  def forward(self, x, h, c):
    
    for n in range(self.n_layers):
      combined = torch.cat((x, h, c), 1)
      ft = torch.sigmoid(self.linearFt(combined))
      it = torch.sigmoid(self.linearIt(combined))
      ct1 = torch.tanh(self.linearCt1(combined))
      c = (ft*c) + (it * ct1)
      #c = (ft*c) + (1-ft) * ct1
      c = self.drop(c)

      ot = torch.sigmoid(self.linearOt(combined))
      h = ot * torch.tanh(c)
      h = self.drop(h)

      out = self.softmax(self.linearO(combined))
    
    return out, h, c

  
class GRU(nn.Module):
  
  def __init__(self, hidden, output_size, input_size):
    super(GRU, self).__init__()
    
    use_bias = True
    
    self.linearZt = nn.Linear(input_size + hidden, hidden, bias=use_bias)
    self.linearRt = nn.Linear(input_size + hidden, hidden, bias=use_bias)
    self.linearHt = nn.Linear(input_size + hidden, hidden, bias=use_bias)
    
    self.normH = nn.BatchNorm1d(input_size + hidden)
    self.drop1 = nn.Dropout(p=0.50)
    self.drop2 = nn.Dropout(p=0.25)
    
    self.linearO = nn.Linear(input_size + hidden, output_size, bias=use_bias)
    self.normO = nn.BatchNorm1d(input_size + hidden)
    
    self.softmax = nn.LogSoftmax(dim=1)
    
  def forward(self, x, h):
    combined = torch.cat((x, h), 1)
    zt = torch.sigmoid(self.linearZt(combined))
    zt = self.drop1(zt)
    rt = torch.sigmoid(self.linearRt(combined))
    rt = self.drop1(rt)
    combined2 = torch.cat((x, rt*h), 1)
    ht = torch.tanh(self.linearHt(combined2))
    ht = self.drop1(ht)
    h = (1-zt) * h + zt * ht
    h = self.drop2(h)
    
    out = self.linearO(combined)
    out = self.softmax(out)
  
    return out, h
  
windows = 6
input_size = train_data.shape[1]
hidden = 8
n_layers = 1
K = 3
learning_rate = 0.05
epochs = 8000
train_loss = []
test_loss = []


model = LSTM(hidden, K, input_size, n_layers)
model.cuda()
loss_fn = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100, 200, 400, 800, 1600, 3200, 6400], gamma=0.1)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=200, gamma=0.1)
N = len(train_data)-(windows+1)
batch_size = 32

x_train = DataLoader(train_data, batch_size=batch_size, shuffle=False, pin_memory=True)
y_train = DataLoader(train_target, batch_size=batch_size, shuffle=False, pin_memory=True)

for e in range(epochs):
    model.train()
    running_loss = 0.0
    testing_loss = 0.0
    
    for i in range(N):
      
    #for i in range(batch_size):
      
        index = np.random.randint(0, N)
        
        #inputs = train_data[i:i+windows]
        #y = train_target[(i+windows)-1]
        #
        
        inputs = train_data[index:index+windows]
        #print(inputs)
        
        y = train_target[(index+windows)-1]
        y_tensor = torch.tensor(y, dtype=torch.long).cuda()

        #print(i, ' Input: ', inputs, ' Target: ', y, ' ', train_data[windows-1])

        h = torch.zeros([1, hidden], dtype=torch.float32).cuda()
        c = torch.zeros([1, hidden], dtype=torch.float32).cuda()

        #print('y: ', y_tensor)

        optimizer.zero_grad()

        for w in range(len(inputs)):
          #print(w)
          x = torch.tensor(inputs[w].reshape(1,-1), dtype=torch.float32).cuda()
          out, h, c = model(x, h, c)

        #print('out: ', out)
        loss = loss_fn(out, y_tensor.unsqueeze(0))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
    run_loss = running_loss / N
    train_loss.append(run_loss)
    scheduler.step()

    model.eval()
    test_count = 100
    for j in range(test_count):
        index = np.random.randint(0, len(test_data)-(windows+1))
        #print('index: ', index)
        h = torch.zeros([1, hidden], dtype=torch.float32).cuda()
        c = torch.zeros([1, hidden], dtype=torch.float32).cuda()
        
        inputs = test_data[index:index+windows]
        y = test_target[(index+windows)-1]
        y_tensor = torch.tensor(y, dtype=torch.long).cuda()

        for w in range(len(inputs)):
            x = torch.tensor(inputs[w].reshape(1,-1), dtype=torch.float32).cuda()
            out, h, c = model(x, h, c)

        loss = loss_fn(out, y_tensor.unsqueeze(0))
        testing_loss += loss.item()
    
    avg_loss = testing_loss / test_count
    test_loss.append(avg_loss)
    
    if e % 10 == 0:
        print(e, ' Train Loss: ', run_loss, ' Test Loss: ', avg_loss)

1740  Train Loss:  0.9132045409308864  Test Loss:  1.0699397844076157
1750  Train Loss:  0.9233357320655831  Test Loss:  1.0914690434932708
1760  Train Loss:  0.9365025439389156  Test Loss:  0.9963647955656052
1770  Train Loss:  0.908079128933343  Test Loss:  1.0304762428998948
1780  Train Loss:  0.9584344475547228  Test Loss:  1.0123466813564301
1790  Train Loss:  0.8779223944329045  Test Loss:  0.9995336127281189
1800  Train Loss:  0.9193265275355497  Test Loss:  0.9730107337236404
1810  Train Loss:  0.9123293926257541  Test Loss:  1.036050118803978
1820  Train Loss:  0.9258117510619095  Test Loss:  1.1024001961946488
1830  Train Loss:  0.9048807621002197  Test Loss:  1.0770631867647171
1840  Train Loss:  0.8609009837628874  Test Loss:  0.986560692191124
1850  Train Loss:  0.928529107131841  Test Loss:  1.0476224154233933
1860  Train Loss:  0.8955481730112025  Test Loss:  1.1205168014764786
1870  Train Loss:  0.9292904150449425  Test Loss:  1.0904557812213898
1880  Train Loss:  0.902

In [0]:
plt.plot(train_loss, label='Train')
plt.plot(test_loss, label='Test')
plt.legend()
plt.show()

In [0]:
y_tensor.cpu().detach().numpy()[0][0]

2